In [1]:
import numpy as np
import pandas as pd
import random as rnd
import plotly.offline as plyo
import cufflinks as cl
from datetime import datetime, timedelta

#initialize variables
iprice=100      #first market price
iprds=100       #days to cover (from today backwards)
istdev=0.02     #standard deviation

#random walk generator
def randomwalk(price, stdev, prds):
    prices=[]
    for i in range(1,prds+1):
        price = price + price*rnd.normalvariate(0,stdev)
        low = price - price*abs(rnd.normalvariate(0,stdev))
        high = price + price*abs(rnd.normalvariate(0,stdev))
        prices.append([price,low, high])
    return(prices)

#function call with initialized variables, directly creating numpy array
randvals = np.array(randomwalk(iprice,istdev,iprds))

#create dataframe from array data
iindex=pd.date_range(datetime.today() - timedelta(days=iprds), periods=iprds, freq='D')
df=pd.DataFrame(randvals, index=iindex, columns=('Open','Low','High'))

#add closing price and modify high/wow if necessary
df['Close']=df['Open'].shift(-1)
df['High']=np.maximum(df['High'], df['Close'])
df['Low']=np.minimum(df['Low'],df['Close'])
df=df.dropna()

#plot data
plyo.init_notebook_mode(connected=True)     #initialize local notebook mode
qf = cl.QuantFig(df)                        #candlestick chart
qf.add_atr(periods=14)                      #add Average True Range Indicator
qf.add_ema(periods=14)                      #add Exponential Moving Average Indicator
plyo.iplot(qf.iplot(asFigure=True))         #execute plot